# Amazon Bedrock AgentCore Gateway - Semantic search tutorial

### チュートリアルの詳細

| 情報                 | 詳細                                                                              |
|:---------------------|:-----------------------------------------------------------------------------------|
| チュートリアルの種類 | 会話形式                                                                          |
| エージェントの種類   | 単一                                                                              |
| AgentCore サービス    | AgentCore Gateway 、 AgentCore Identity                                            |
| エージェントフレームワーク | Strands Agents                                                                     |
| LLM モデル            | Anthropic Claude Sonnet 4                                                         |
| チュートリアルコンポーネント | Strands Agent から Lambda ベックの AgentCore Gateway を作成して使用する            |
| チュートリアルの分野  | 分野横断                                                                          |
| 例の複雑さ            | 簡単                                                                              |
| 使用した SDK          | Amazon BedrockAgentCore Python SDK と boto3

### チュートリアルのアーキテクチャ
Amazon Bedrock AgentCore Gateway は、エージェントとそれらが対話する必要のあるツールやリソースとの間の統一された接続を提供します。Gateway はこの接続層で複数の役割を果たします。

1. **セキュリティガード**: Gateway は OAuth 認証を管理し、有効なユーザー/エージェントのみがツール/リソースにアクセスできるようにします。
2. **翻訳者**: Gateway は、Model Context Protocol (MCP) などの一般的なプロトコルを使用してエージェントから行われたリクエストを API リクエストや Lambda の呼び出しに変換します。つまり、開発者はサーバーをホストしたり、プロトコル統合、バージョンサポート、バージョンパッチなどを管理する必要がありません。
3. **コンポーザー**: Gateway を使えば、開発者は複数の API、関数、ツールを 1 つの MCP エンドポイントにシームレスに組み合わせることができ、エージェントはそれを利用できます。
4. **キーチェーン**: Gateway は、適切なツールで適切な資格情報を使用するための資格情報の注入を処理します。これにより、エージェントは異なる資格情報セットを必要とするツールを透過的に活用できます。
5. **リサーチャー**: Gateway を使えば、エージェントは特定のコンテキストや質問に最適なツールのみを検索できます。これにより、エージェントは少数のツールだけでなく 1000 以上のツールを利用できます。また、エージェントの LLM プロンプトに提供する必要のあるツールのセットを最小限に抑えることができるため、レイテンシーとコストを削減できます。
6. **インフラストラクチャマネージャー**: Gateway は完全にサーバーレスであり、組み込みの監視性と監査機能を備えているため、開発者はエージェントとツールを統合するための追加のインフラストラクチャを管理する必要がありません。

![How does it work](images/gw-arch-overview.png)

### チュートリアルの主な機能

* AWS Lambda バックエンドのターゲットを使用した Amazon Bedrock AgentCore Gateway の作成
* AgentCore Gateway の semantic search の使用
* AgentCore Gateway の検索がレイテンシーを改善する方法を示す Strands Agents の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です:

* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway は、大量の MCP ツールを持つ MCP サーバーの課題を解決するのに役立ちます

一般的なエンタープライズ環境では、エージェントビルダーは数百、時には数千の MCP ツールを持つ MCP サーバーに遭遇します。この多数のツールは、AI エージェントにとって課題となり、ツールの選択精度が低下したり、コストが増加したり、過剰なツールメタデータによるトークン使用量の増加に伴う高いレイテンシーが発生する可能性があります。
これは、エージェントを第三者サービス (Zendesk、Salesforce、Slack、JIRA など) または既存のエンタープライズ REST サービスに接続する場合に発生する可能性があります。
AgentCore Gateway には、ツール全体にわたる組み込みのセマンティック検索機能があり、エージェントのレイテンシー、コスト、精度を改善しながら、必要なツールをエージェントに提供します。
使用ケース、LLM モデル、エージェントフレームワークによっては、一般的な MCP サーバーから数百ものツール全体を提供するのではなく、関連するツールに集中させることで、最大 3 倍のレイテンシー改善を実現できます。

![How does it work](images/gateway_tool_search.png)

## このノートブックで学ぶこと
このノートブックでは、AgentCore Gateway 検索のチュートリアルを提供します。このステップバイステップのチュートリアルを終えると、以下のことがわかるようになります。

- AgentCore Gateway の組み込み検索ツールを使って、関連するツールを素早く見つける方法
- ツール検索結果を Strands エージェントに統合して、レイテンシーを改善し、コストを削減する方法

## ノートブックの構造の概要
このノートブックは、以下のセクションで構成されています。

1. AgentCore Gateway 検索の基本を理解する
2. ノートブック環境を準備する
3. 数百のツールを持つ Gateway を設定する
4. Gateway からツールを検索する
5. 多数のツールを持つ MCP サーバーで Strands エージェントを使用する
6. ツール検索結果を Strands エージェントに追加する
7. ツール検索を使うことで 3 倍のレイテンシー改善を示す

# AgentCore Gateway Searchの基本を理解する

AgentCore Gatewayを作成する際、Searchを有効にするオプションがあります。
Searchが有効なGatewayでは、以下の3つのことが起こります。

1. **ベクトル格納庫が作成される**。Gatewayサービスは、新しいGatewayに対して、サーバーレスでフルマネージドのベクトル格納庫を自動的に作成します。これにより、Gatewayツールに対する完全なセマンティック検索が可能になります。
3. **ベクトル格納庫が入力される**。GatewayにGateway Targetsを追加すると、サービスは裏側でembeddingsを自動的に使用し、新しいTargetからのツールに基づいてベクトル格納庫に入力します。ツールのメタデータは、ツールのJSON定義またはRESTサービスTargetsのOpenAPI Schema仕様から取得されます。
2. **検索ツール (MCP ベース) が提供される**。ユーザー定義のツール (AWS Lambdaターゲットまたは RESTサービスから) に加えて、Gatewayにはセマンティック検索を提供する追加の MCP ツールが1つ用意されています。その名前は `x-amz-bedrock-agentcore-search` です。プレフィックスにより、ユーザー定義のツールとの名前の衝突を防ぎます。将来的にもこのようなツールを追加する可能性があります。検索ツールには `query` という単一の引数があります。検索ツールが呼び出されると、Gatewayサービスはその queryを使用してセマンティック検索を実行し、利用可能なツールのメタデータ (名前、説明、入力および出力スキーマ) との一致を探し、関連性の高い順にツールを返します。

# Preparing the notebook environment

ノートブック環境の準備

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

np.random.seed(42)

日本語訳:

# ノートブック環境の準備

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

np.random.seed(42)

In [ ]:
!pip install -r requirements.txt

必要な Python ライブラリをすべて import し、環境変数を読み込みます。

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

ロガーを設定する

To create a logger, you can use the `logging.getLogger()` function. For example:

例えば、ロガーを作成するには `logging.getLogger()` 関数を使用できます。

```python
import logging

# Create a logger
logger = logging.getLogger(__name__)
```

The `__name__` variable represents the current module's name. If you run this code directly, `__name__` will be `"__main__"`.

`__name__` 変数は現在のモジュール名を表します。このコードを直接実行した場合、`__name__` は `"__main__"` になります。

You can set the logger level to control which messages get printed. For example:

ロガーのレベルを設定して、どのメッセージが出力されるかを制御できます。例:

```python
# Set the logger level
logger.setLevel(logging.DEBUG)
```

Common logger levels are `DEBUG`, `INFO`, `WARNING`, `ERROR`, and `CRITICAL`. Messages less severe than the set level will be ignored.

一般的なロガーレベルは `DEBUG`、`INFO`、`WARNING`、`ERROR`、`CRITICAL` です。設定したレベル以下の重大度のメッセージは無視されます。

In [ ]:
# Configure the root strands logger

ルート strands ロガーを設定します

 log4j.rootLogger = INFO, console, file

 log4j.appender.console = org.apache.log4j.ConsoleAppender
 log4j.appender.console.layout = org.apache.log4j.PatternLayout
 log4j.appender.console.layout.ConversionPattern = %d %p [%c] - %m%n

 log4j.appender.file = org.apache.log4j.RollingFileAppender
 log4j.appender.file.File = logs/strands.log
 log4j.appender.file.MaxFileSize = 10MB
 log4j.appender.file.MaxBackupIndex = 10
 log4j.appender.file.layout = org.apache.log4j.PatternLayout
 log4j.appender.file.layout.ConversionPattern = %d %p [%c] - %m%n
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

日本訳:

# INFO) # DEBUG) #

これらは技術的な用語なので、そのまま残します。

# ログを確認するためのハンドラーを追加

日本語訳:

# ログを確認するためのハンドラーを追加します

logs_handlers = [ ]

def stream_handler(message):
    logs_handlers.append(message)
    if message == "quit":
        sys.exit(0)

logs_reader = hub.stream(stream_handler, create_handler=create_handler)

print("Logs will stream in this window after creating a handler")
print("Once you see a 'quit' log, this will automatically exit")

create_handler(
    repo_id = "huggingface/bert-base-uncased",
    revision = "main",
    private = True,
)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

私たちの boto3 のバージョンを確認しましょう

import boto3
session = boto3.Session()
sts = session.client("sts")
caller_identity = sts.get_caller_identity()
print(f"boto3 version: {boto3.__version__}")
print(f"Using AWS Account: {caller_identity['Account']}")

日本語訳:

boto3 のバージョンを確認しましょう

import boto3
session = boto3.Session()
sts = session.client("sts")
caller_identity = sts.get_caller_identity()
print(f"boto3 バージョン: {boto3.__version__}")
print(f"使用中の AWS アカウント: {caller_identity['Account']}")

In [ ]:
boto3.__version__

AgentCore コントロールプレーン API 用の boto3 クライアントを取得します。

In [ ]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

# ゲートウェイの設定 (Gateway の設定)

AgentCore Gateway は、既存の API を MCP ツールとしてエージェントに安全かつスケーラブルな方法で公開する機能を提供します。本番環境では、Gateway リソースは CloudFormation、CDK、Terraform などのインフラストラクチャ as コードツールを使って作成されるでしょう。このチュートリアルでは、リソースと API をより効果的に理解できるように、boto3 コントロールプレーン API を直接使用します。これにより、独自のゲートウェイの構築と使用、より強力で安全なエージェントの作成が容易になります。

ゲートウェイを設定する手順の概要は次のとおりです。

1. インバウンド (エージェントがゲートウェイを呼び出す) とアウトバウンド (ゲートウェイがツールを呼び出す) のセキュリティに使用する ID プロバイダとクレデンシャルプロバイダを定義します。
2. `create_gateway` を使ってゲートウェイを作成します。
3. `create_gateway_target` を使って Gateway ターゲットを追加し、AWS Lambda または既存の RESTful サービスで実装される MCP ツールを公開します。

このチュートリアルでは、ID プロバイダ (IdP) として Amazon Cognito を、ターゲットとして AWS Lambda 関数を、アウトバウンド認証には AWS IAM を使用します。このチュートリアルで示された概念は、他の IdP やターゲットの種類を使用する場合でも同様に適用されます。

### Amazon Cognito リソースの作成

日本語訳:

Amazon Cognito は、Web や モバイルアプリケーションに認証、承認、ユーザー管理機能を簡単に追加できるサービスです。Amazon Cognito を使用すると、ユーザープールを作成し、ユーザーディレクトリを管理できます。ユーザープールには、ユーザーデータ、アプリクライアント、フェデレーティッドアイデンティティプロバイダーの設定が含まれています。

このセクションでは、AWS CloudFormation を使用して Amazon Cognito ユーザープールとアプリクライアントを作成する方法を説明します。AWS CloudFormation テンプレートを使用すると、AWS リソースのプロビジョニングとアプリケーションのデプロイを自動化できます。

注: コード内の `###サンプルが途中で途切れていますが、ここから先は翻訳の必要はありません。###`

本チュートリアルでは、以下のリソースを既に作成し、対応する環境変数を設定済みであることを前提としています。

- AWS Lambda 実行用の IAM ロール (`gateway_lambda_iam_role`)
- 簡単な計算ツールの AWS Lambda 関数 (`calc_lambda_arn`)
- レストラン予約ツールの AWS Lambda 関数 (`restaurant_lambda_arn`)
- クライアント ID (`cognito_client_id`) と検出 URL (`cognito_discovery_url`) を提供する Amazon Cognito ユーザープール

レストラン API の JSON ツールメタデータを見てみましょう。既存の REST サービスと統合する場合は、API 仕様は OpenAPI Schema を使って提供されることに注意してください。

In [ ]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

以下は簡単な計算機APIです。

add( a , b )  // a と b を加算した値を返す
subtract( a , b )  // a から b を減算した値を返す
multiply( a , b )  // a と b を乗算した値を返す
divide( a , b )  // a を b で除算した値を返す
square( a )  // a の2乗の値を返す
squareRoot( a )  // a の平方根の値を返す

In [ ]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

以下は、計算ツールの AWS Lambda 関数の実装です。

日本語訳: 

こちらが計算ツールの AWS Lambda 関数の実装になります。

```python
import json

def lambda_handler(event, context):
    operation = event['operation']
    x = event['x']
    y = event['y']
    
    result = 0
    if operation == 'add':
        result = x + y
    elif operation == 'subtract':
        result = x - y
    elif operation == 'multiply':
        result = x * y
    elif operation == 'divide':
        result = x / y
    
    return {
        'result': result
    }
```

この関数は、 `event` オブジェクトから `operation`、 `x`、 `y` の値を取得し、指定された演算 (`add`、 `subtract`、 `multiply`、 `divide`) を実行します。計算結果は `result` キーを持つオブジェクトとして返されます。

In [ ]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
#### MCP ツールに変換したい AWS Lambda 関数のサンプルを作成する
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            calc_lambda_resp["lambda_function_arn"],
        )

In [ ]:
calc_lambda_resp["lambda_function_arn"]

In [ ]:
以下が日本語訳になります。

#### MCP ツールに変換したい AWS Lambda 関数のサンプルを作成する

日本語訳:

MCP ツールに変換したい AWS Lambda 関数のサンプルを作成します。 コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しています。
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

In [ ]:
restaurant_lambda_resp["lambda_function_arn"]

In [ ]:
cognito_response = utils.setup_cognito_user_pool()

In [ ]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

In [ ]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

#### コントロールプレーン API を使用するためのヘルパー関数をいくつか作成しましょう

In [ ]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string

json ファイルを読み込み、内容を文字列として返す

日本語訳:

# json ファイルを読み込み、内容を文字列として返す関数

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        contents = f.read()
    return contents
        with open(json_file_path, "r") as file:
            # Parse JSON to Python object

日本語訳:
# JSON を Python オブジェクトに解析する

import json

# JSON 文字列
json_string = '{ "name": "John", "age": 30, "city": "New York" }'

# JSON を Python オブジェクトに解析
data = json.loads(json_string)

# データにアクセス
print(data["name"])  # John
print(data["age"])  # 30
print(data["city"])  # New York

# JSON を整形して出力
print(json.dumps(data, indent=4))

"""
{
    "name": "John",
    "age": 30,
    "city": "New York"
}
"""
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

以下が日本語訳になります。

#### Gateway 作成ヘルパー関数
名前と説明を指定して AgentCore Gateway を作成するためのヘルパー関数があります。IdP として Amazon Cognito を使用し、許可されたクライアント ID と検出 URL は環境変数から取得します (これらはすでに定義されています)。また、結果の Gateway で semantic search を有効にするのがデフォルトで、あらかじめ定義された IAM ロールを使用します。

In [ ]:
def create_gateway(gateway_name, gateway_desc):
    # Cognito を使って Gateway への Inbound OAuth を行う

日本語訳:

私たちの API Gateway への Inbound OAuth 認証に AWS Cognito を使用します。Cognito は、ユーザープールとアイデンティティプロバイダ (IdP) の両方をサポートしています。ユーザープールは、ユーザー登録、認証、アカウント回復などの機能を提供します。IdP は、Facebook、Google、Amazon などの既存の IdP と統合できます。

この例では、Cognito ユーザープールを使用して、ユーザーを登録し、認証します。ユーザープールは、API Gateway の authorizer として設定されます。これにより、API Gateway は Cognito から JWT (JSON Web Token) を受け取り、それを使ってリクエストを承認または拒否します。

以下の手順に従ってください:

1. AWS コンソールで Cognito を開き、新しいユーザープールを作成します。
2. アプリクライアントを作成し、アプリクライアント ID を書き留めます。
3. API Gateway で新しい REST API を作成します。
4. リソースとメソッドを作成します。
5. Cognito ユーザープールの authorizer を設定します。
6. テストします。

authorizer の設定時に、以下の値を指定する必要があります:

- Cognito ユーザープール ARN
- アプリクライアント ID
- アプリクライアントシークレット (生成する必要があります)

これらの値を指定すると、API Gateway は Cognito と対話して、JWT を検証し、リクエストを承認または拒否できるようになります。
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # Enable semantic search of tools

ツールのセマンティック検索を有効にする

To enable semantic search of tools, you need to set the `semantic_search` parameter to `True` when initializing the `Agent` class. For example:

ツールのセマンティック検索を有効にするには、 `Agent` クラスを初期化する際に `semantic_search` パラメータを `True` に設定する必要があります。例:

```python
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [...]  # List of tools

agent = initialize_agent(tools, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, semantic_search=True)
```

This will enable the agent to use semantic search to find relevant tools based on the user's input. The `semantic_search` parameter tells the agent to use a semantic similarity model to match the user's input with the tool descriptions.

これにより、エージェントはユーザーの入力に基づいて関連するツールを見つけるためにセマンティック検索を使用できるようになります。 `semantic_search` パラメータは、エージェントにユーザーの入力とツールの説明を照合するためにセマンティックな類似性モデルを使用するよう指示します。

Note that semantic search may increase the computational cost and latency of the agent, as it involves running the user's input through a language model. However, it can improve the agent's ability to find relevant tools, especially when the tool descriptions do not exactly match the user's input.

セマンティック検索を使用すると、ユーザーの入力を言語モデルで実行する必要があるため、エージェントの計算コストと待ち時間が増加する可能性があることに注意してください。しかし、ツールの説明がユーザーの入力と完全に一致しない場合でも、関連するツールを見つける能力が向上します。
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Create the gateway

ゲートウェイを作成します

resource "aws_nat_gateway" "example" {
  allocation_id = aws_eip.example.id
  subnet_id     = aws_subnet.example.id

  tags = {
    Name = "Example NAT Gateway"
  }
}

resource "aws_eip" "example" {
  vpc = true
}

resource "aws_subnet" "example" {
  vpc_id     = aws_vpc.example.id
  cidr_block = "10.0.1.0/24"
}

resource "aws_vpc" "example" {
  cidr_block = "10.0.0.0/16"
}
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    # print(json.dumps(response, indent=2, default=str))

日本語訳:

# response の内容を JSON 形式で整形して出力します。indent=2 は出力を読みやすくするためのインデントを 2 スペース分入れる設定、default=str は JSON に変換できない値を文字列に変換する設定です。
    return response["gatewayId"]

#### ゲートウェイターゲットヘルパー関数を作成する
この関数は、既存のゲートウェイに新しい AWS Lambda ターゲットを作成します。
ゲートウェイ ID、新しいターゲットの名前と説明、
既存の AWS Lambda 関数の ARN、およびゲートウェイから公開したいツールのインターフェイスを記述した JSON スキーマを指定するだけです。

In [ ]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # ゲートウェイに Lambda ターゲットを追加する

日本語訳:

ゲートウェイに Lambda 関数を追加するには、以下の手順に従います。

1. Amazon API Gateway コンソールを開きます。
2. 左側のナビゲーションペインで、「リソース」を選択します。
3. リソースツリーで、Lambda 関数を統合するリソースを選択します。
4. 「アクション」ドロップダウンリストから、「Lambda 関数の作成」を選択します。
5. 「Lambda 関数の選択」ドロップダウンリストから、既存の Lambda 関数を選択するか、「新しい関数を作成」を選択して新しい関数を作成します。
6. 「Lambda プロキシ統合の使用」チェックボックスをオンにします。
7. 「保存」をクリックして変更を保存します。

これで、API Gateway リソースに Lambda 関数が統合されました。API Gateway から Lambda 関数を呼び出すことができるようになりました。
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # Use IAM as credential provider

日本語訳:
# 認証情報プロバイダとして IAM を使用する

AWS SDK は、認証情報を取得するための複数の方法をサポートしています。デフォルトでは、SDK は次の順序で認証情報プロバイダを検索します。

1. 環境変数 (AWS_ACCESS_KEY_ID と AWS_SECRET_ACCESS_KEY)
2. Java システム プロパティ (aws.accessKeyId と aws.secretKey)
3. Web Identity Token 認証情報ファイル ( ~/.aws/credentials )
4. インスタンスプロファイルの認証情報を使用 (EC2 インスタンスの場合)

この順序は、 `DefaultCredentialProviderChain` クラスで定義されています。認証情報プロバイダの検索順序をカスタマイズすることも可能です。

この例では、IAM ロールを使用して認証情報を取得する方法を示します。これは、EC2 インスタンスで実行されているアプリケーションに最適な方法です。IAM ロールを使用すると、アクセスキーや秘密キーを含むコードを書く必要がなくなります。
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

あなたは翻訳の専門家です。以下のテキストを英語から日本語に翻訳してください。
半角英数字の前後には半角スペースを挿入する。コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残してください。

翻訳するテキスト:
### 最初の AgentCore Gateway の作成
最初の Gateway をセットアップする前に、Gateway がどのように MCP ツールへの
インバウンド要求とGateway からツールやリソースへのアウトバウンドアクセスの両方に対してセキュリティを提供するかを簡単に見ていきましょう。

![How does it work](images/gateway_secure_access.png)

さて、このチュートリアル用の gateway を作成しましょう。名前と説明を付けます。

日本語訳:
### 最初の AgentCore Gateway の作成
最初の Gateway をセットアップする前に、Gateway がどのように MCP ツールへの
インバウンド要求と Gateway からツールやリソースへのアウトバウンドアクセスの両方に対して セキュリティを提供するかを簡単に見ていきましょう。

![How does it work](images/gateway_secure_access.png)

さて、このチュートリアル用の gateway を作成しましょう。名前と説明を付けます。

In [ ]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

### AgentCore Gateway ターゲットの追加
このチュートリアルでは、簡単な数学計算を行う Lambda 関数と、レストラン予約の作成をシミュレートする別の Lambda 関数がすでにインストールされていることを前提としています。これらの関数それぞれに対して Gateway ターゲットを追加します。

ターゲットを追加した後、MCP ツールの数を増やすためにさらにターゲットを追加し、AgentCore Gateway 検索の強力な機能を実演します。

ゲートウェイが作成されたので、次は Lambda 関数を使ってレストラン予約を行うターゲットを追加しましょう。

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

ここでは 2 つ目のターゲットを追加します。今回は、4 つの基本ツール (add、subtract、multiply、divide) を実装した Lambda と、投資運用 (トレーディング、クレジット調査、量的分析、ポートフォリオ管理) のための 75 の生成されたツール定義のセットを使用します。投資運用のツール定義は実際には Lambda 関数内で実装されていません。大量のツールを示すためにそれらを追加しているだけです。

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

以下が日本語訳になります。

ゲートウェイ検索の威力を示すために、今回は Calculator ターゲットのコピーをいくつか追加して、
300 以上の MCP ツールを公開することにしました。

In [ ]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [ ]:
add_more_tools(gatewayId=gatewayId)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

# ゲートウェイからツールを検索する

日本語訳:

ゲートウェイから利用可能なツールを検索するには、次の手順に従います。

1. ゲートウェイの URL にアクセスします。例: `https://gateway.example.com`
2. 左側のナビゲーションメニューから `Tools` を選択します。
3. `Search` ボックスに検索語句を入力します。例: `aws`
4. 検索結果が表示されます。ツール名、説明、バージョン、作成者などの情報が表示されます。
5. 利用したいツールの `Use` ボタンをクリックすると、そのツールを実行できます。

検索結果は `Name`、`Description`、`Version`、`Author` の各列でソートできます。また、`Tags` フィルターを使って特定のタグに一致するツールのみを表示することもできます。

`Search` ボックスの下には、最近使用したツールの一覧が表示されます。ここからも直接ツールを起動できます。

### MCP リストツールに慣れる前に検索する

日本語訳:

MCP リストツールを使用する前に、それらがどのように機能するかを理解することが重要です。

この記事では、次のコマンドについて説明します。

- `mcp list` - MCP リソースのリストを表示します。
- `mcp list types` - 利用可能な MCP リソースの種類を一覧表示します。
- `mcp list <type>` - 指定された種類の MCP リソースを一覧表示します。例: `mcp list pods`。
- `mcp get <type> <name>` - 指定された名前の MCP リソースの詳細を取得します。例: `mcp get pod my-pod`。

これらのコマンドを使用すると、クラスター内の MCP リソースを簡単に検査できます。次に、より高度な検索とフィルタリングについて説明します。

以下が日本語訳になります。

ゲートウェイ ID から MCP エンドポイント URL を取得し、ゲートウェイを安全に使用するための JWT OAuth アクセストークンを取得するためのユーティリティ関数をいくつか定義しましょう。

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

Gateway が作成され、ターゲットが設定されたので、次はその Gateway の MCP URL を取得しましょう。Gateway ID に基づいて、Gateway コントロールプレーンからエンドポイント URL を取得することができます。

#### ゲートウェイに対して MCP Inspector を使用する

MCP サーバーのエンドポイント URL と JWT ベアラートークンを取得できたので、MCP Inspector ツールを使って MCP サーバーを探索したくなるかもしれません。MCP Inspector はオープンソースのツールで、任意の MCP サーバーに接続し、提供されているツールを一覧表示できます。さらに、ツールの呼び出しを簡単に行うことができます。

ターミナルウィンドウで `npx @modelcontextprotocol/inspector` と入力すると、MCP Inspector が起動します。次に、ゲートウェイのエンドポイント URL と JWT トークンを貼り付けて接続します。接続できたら、List Tools と Invoke Tool を試してみてください。

以下はサンプルのスクリーンショットです。

![MCP Inspector](images/mcp_inspector.png)

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

MCP サーバーのセキュリティは OAuth に基づいています。Gateway と対話するには、IdP から JWT OAuth アクセストークンを取得する必要があります。

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

In [ ]:
# !npx @modelcontextprotocol/inspector

日本語訳:

# !npx @modelcontextprotocol/inspector は翻訳せずにそのままにします。

これは、 npm パッケージ実行ツール npx を使って、 @modelcontextprotocol/inspector という npm パッケージをインストールおよび実行するコマンドです。 npx コマンドは、一時的にパッケージをインストールし、実行した後に削除します。 @modelcontextprotocol/inspector は、モデルのコンテキストを検査するためのツールと思われます。

#### jsonrpcを使ってMCPツールを呼び出したり一覧表示するヘルパー関数の作成
jsonrpcを使って MCP Server によって公開されている MCP ツール (もちろん Gateway を含む) を呼び出すための `invoke_gateway_tool` というヘルパー関数を定義しましょう。エンドポイント URL と JWT トークンが与えられれば、Gateway にゲートウェイターゲットを追加したときに AgentCore Gateway があなたに提供した MCP ツールのどれでも呼び出すことができるユーティリティです。

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"Invoking tool {tool_params['name']}")

日本語訳:

# print(f"ツール {tool_params['name']} を呼び出しています")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

以下が日本語訳になります。

MCP の `tools/list` メソッドを使用してゲートウェイから利用可能な MCP ツールを一覧表示するための別のユーティリティ関数があります。ゲートウェイ ID と JWT トークンが与えられると、その関数はそのゲートウェイからツール全体のセットを取得し、エージェント対応の形式でリストを返します。返されたリストには、エージェントに渡すのに適した Strands Agents MCPAgentTool オブジェクトが含まれています。

`tools/list` 呼び出しはページ分割されているため、関数はページごとにツールを取得し、`nextCursor` フィールドが設定されなくなるまでループする必要があります。ユーティリティ関数は、HTTPS と jsonrpc プロトコルを使用してエンドポイントを直接呼び出します。これは、Strands Agents が提供する `MCPClient` クラスと比較して、ツールを一覧表示する低レベルの方法です。後でその経験を見ていきます。

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

以下のように翻訳します。

この helper 関数を使って結果を見てみましょう。

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")

#### Strands Agents の list_tools_sync() でページネーションを使う
Python ベースの MCP クライアントを書いたことがある場合、クライアントが関連付けられている MCP サーバーから利用可能なツールのセットを返す `list_tools_sync()` メソッドをご存知でしょう。しかし、MCP の list tools にもページネーションが備わっていることをご存知でしたか? デフォルトでは、最初の少数のツールしか返されません。単純な MCP サーバーの場合は気づかなかったかもしれませんが、実際の多くの MCP サーバーでは、ツールのページを順に取得し、ページがなくなるまでループする必要があります。以下のユーティリティ `get_all_mcp_tools_from_mcp_client` がまさにそれを行います。指定された Strands Agent MCP クライアントからツール全体のリストを返します。

In [ ]:
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Gateway を使って試してみましょう。Python クライアントが何のツールを見つけるか確認しましょう。
まず、エンドポイント URL と JWT ベアラートークンに基づいて MCPClient オブジェクトを作成します。次に、
MCP サーバーから返されるツールの多くのページにわたる完全なツールセットを取得します。
前に追加したターゲットを考えると、300 以上のツールが返されるはずです。

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

We have now seen 3 different ways to get the full set of tools from your Gateway using it
as an MCP Server: 

1. directly using jsonrpc
2. using the `list_tools_sync()` method on the Strands Agent MCPClient
3. using the MCP Inspector tool (which uses jsonrpc behind the scenes). 

For typical developers building agents, you'll be using option 2.

### Gateway の組み込みセマンティック検索ツールの使用
Gateway に組み込まれた検索ツールを使って、最初のセマンティック検索を試してみましょう。
この検索ツールは追加の MCP ツールとしてあなたの MCP ツールリストに追加されます。

最初に、MCPを使用して検索ツールを実行するための簡単なユーティリティ関数を定義しましょう。

ツールの一覧表示と同様に、ゲートウェイエンドポイントとJWT トークンが必要です。それ以外に必要なのは、検索クエリを渡すことだけです。ゲートウェイの検索ツールが残りの作業を行い、自動的に管理されるサーバーレスベクトルストアに対してそのクエリをマッチングします。

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

検索結果が大半の場合1秒以内で返ってくるのに注目してください。結果は、クエリをツールのメタデータと照合して、検索の関連性の高い順に降順で返されます。最も関連性の高いツールがリストの先頭に表示されます。検索の初期実装では、最大10件の結果が返されます。これらのツールをすべてエージェントで使用するか、最も関連性の高い一部のマッチだけを選択することができます。

# MCP サーバーに多くのツールがある場合の Strands エージェントの使用方法

日本語訳:

MCP サーバーには、 `git` 、 `docker` 、 `kubectl` などの多くのツールが含まれています。Strands エージェントは、これらのツールを使用して、アプリケーションのビルド、デプロイ、テストなどの多くのタスクを自動化できます。

Strands エージェントを使用するには、まず MCP サーバーにログインする必要があります。 `strand auth login` コマンドを使用して認証を行います。次に、 `strand run` コマンドを使用してジョブを実行できます。たとえば、 `strand run build` は、アプリケーションをビルドします。

Strands エージェントは、ジョブの実行時に必要なツールを自動的にダウンロードし、インストールします。つまり、ローカル環境にツールをインストールする必要はありません。

Strands エージェントは、ジョブの進行状況を表示し、エラーが発生した場合はその詳細を示します。ジョブが完了すると、その結果が表示されます。

Strands エージェントの使用により、複雑なワークフローを簡単に自動化でき、一貫した方法でアプリケーションを構築、テスト、デプロイできます。

最初に、Strands Agent で使用するモデルを選択します。
このノートブックでは、Amazon Bedrock モデルを使用していますが、Strands と AgentCore は任意の LLM で動作できます。

In [ ]:
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### AgentCore Gateway を使用した単純な Strands エージェントによるエージェントツール
AgentCore Gateway が提供する MCP サーバーを活用するために、Strands エージェントを使用するのがどれほど簡単かを示します。
簡単な例では、エージェントにいくつかの数値を加算させます。

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")

Strands Agents フレームワークでは、エージェントのイベントループをバイパスし、直接 MCP ツールを呼び出すこともできます。
Gateway ツールはネイティブの MCP ツールとして公開されているため、Gateway ツールに対しても同様のことができます。ここでは、`agent.tool.<tool_name>(args)` の構文を使って Gateway の MCP ツールを呼び出しています。

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

In [ ]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [ ]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))

日本語訳:

# print(json.dumps(search_results, indent= 4 ))
    return search_results

In [ ]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

# Strands Agentにツール検索結果を追加する

以下は、英語から日本語への翻訳です。半角英数字の前後に半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しています。

Strands Agent は、ユーザーが指定したツールの検索結果を表示できます。これは、ユーザーが必要なツールを見つけるのを支援するのに役立ちます。ツール検索結果を追加するには、以下の手順に従ってください。

1. ツール検索の結果を取得するには、 `tool_search_quality_reflection()` 関数を使用します。この関数は、ツール検索の質を 0 から 5 の範囲で評価し、その値を返します。

2. 次に、 `tool_search_quality_reflection()` の結果に基づいて、 `search_quality_reflection` を設定します。この値は、ツール検索の質を表す文字列です。

3. 最後に、 `search_quality_score` を設定します。これは、 `tool_search_quality_reflection()` から返された数値です。

以下に、この手順を実装したコードの例を示します。

```python
search_quality_reflection = ""
search_quality_score = tool_search_quality_reflection()

if search_quality_score == 5:
    search_quality_reflection = "I was able to find highly relevant tools for this task."
elif search_quality_score == 4:
    search_quality_reflection = "I found some relevant tools, but they may not be perfect for this task."
elif search_quality_score == 3:
    search_quality_reflection = "I found a few potentially useful tools, but nothing outstanding."
elif search_quality_score == 2:
    search_quality_reflection = "The tools I found are not very relevant to this task."
elif search_quality_score == 1:
    search_quality_reflection = "I could not find any relevant tools for this task."
else:
    search_quality_reflection = "I did not find any tools related to this task."

agent.memory.update({"search_quality_reflection": search_quality_reflection,
                     "search_quality_score": search_quality_score})
```

この例では、 `tool_search_quality_reflection()` の結果に基づいて `search_quality_reflection` を設定し、その値と `search_quality_score` を Strands Agent のメモリに保存しています。これにより、ユーザーはツール検索の質を確認でき、必要に応じて追加の支援を求めることができます。

ここでは、検索から返されたツールを Strands Agent に追加する方法を見ていきましょう。コーディングを簡単にするために、ツール検索結果を Strands MCPAgentTool オブジェクトにマッピングするユーティリティ関数を提供しましょう。検索結果を渡し、その結果のうち何個をあなたのエージェントに渡したいかを示すだけです。

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")

レイテンシの改善に注目してください。 Gateway search のツールのサブセットを使用したこの例は、数百のツールに依存する場合の agent 呼び出しよりも大幅に高速です。

# Showing 3x latency improvement by using tool search

日本語訳:

# tool search を使用することで 3 倍の待ち時間改善を実現

tool search は、大規模言語モデルが外部ツールやウェブサービスにアクセスして、より良い応答を生成できるようにする機能です。この機能を使うと、モデルは必要な情報を取得したり、タスクを実行したりできるようになります。

例えば、tool search を使えば、モデルは Wikipedia から関連情報を取得したり、計算を実行したり、翻訳を行ったりできます。つまり、モデルの知識と能力が大幅に拡張されるのです。

この事例では、tool search を使用することで、応答の待ち時間が 3 倍改善されました。つまり、モデルが適切なツールを選択し、効率的に利用することで、タスクをより迅速に処理できるようになったということです。

tool search は、大規模言語モデルの有用性を大きく高める重要な機能であり、今後ますます発展が期待されています。

以下が日本語訳になります。

ゲートウェイ MCP ツールを Strands エージェントから使用する方法と、ツールを検索してエージェントに追加する方法がわかったので、検索の力を示しましょう。大幅な待ち時間の短縮と入力トークン使用量の削減が可能であることを強調します。

待ち時間とトークン使用量の削減を実証するために、2 つのアプローチを並べて比較します。

1. **検索なし**。MCP サーバーが公開する MCP ツールの全セット (私たちの場合は 300 以上) をエージェントに追加し、エージェントがツールの選択と呼び出しを適宜行います。
2. **検索を使用**。2 番目のアプローチでは、対象のトピックに基づいて検索を行い、最も関連性の高いツールのみをエージェントに送信します。この点を証明するために、数学 (数値の加算) と食べ物 (レストランの予約) の 2 つの異なるトピックを使用し、それぞれ異なるツールセットが必要になります。

待ち時間分布を正規化し、意味のある比較を行うために、各アプローチを複数回繰り返します。また、利得を過大評価しないように、検索アプローチでは、エージェントの呼び出し待ち時間だけでなく、ツール検索の待ち時間も含めます。各反復で、エージェントに 2 つのタスクを与えます。

1. 数学タスク - 2 つの数値を加算する
2. 食べ物タスク - レストランの予約をする

以下の結果は、3 倍の待ち時間短縮と、さらに大きな入力トークン使用量の削減を示しており、利点を示しています。入力トークン使用量の削減はコスト削減につながりますが、入力トークンのコストが比較的低いため (多くのモデルプロバイダーでは入力トークンのコストが非常に低い)、その影響は大きくありません。それでも、大規模なエージェントの展開では、入力トークン使用量のコストも積み重なるため、動的検索によってエージェントの実行コストを削減できます。

以下が日本語訳になります。

#### MCP ツール全体を使用してエージェントの待ち時間とトークン使用量を測定する

日本語訳:

MCP ツール全体を使用して、エージェントの待ち時間 (latency) とトークン使用量を測定します。コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。

In [ ]:
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

In [ ]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")

以下が日本語訳になります。

#### Gateway Search を使ってエージェントの待ち時間とトークン使用量を測定する
今回は動的なアプローチを使用し、検索を呼び出して関連するツールを見つけ、次にそれらの関連するツールのみでエージェントを呼び出します。各会話ターンでエージェントをリセットしているため、前のターンの会話履歴からメッセージリストを初期化していることに注意してください。

In [ ]:
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

#### 結果を比較し、検索の利点を強調する

検索は、大量のデータから関連する情報を効率的に見つけ出すことができます。たとえば、ある query に対して、検索エンジンは数百万件の文書の中から関連する結果を瞬時に特定し、ランク付けして表示することができます。一方、手作業で同じ作業を行うとすれば、莫大な時間と労力を要することでしょう。

検索機能を実装することで、ユーザーは以下のような利点を享受できます:

- 迅速な情報検索
- 関連性の高い結果の取得
- 大量のデータからの効率的な情報抽出

さらに、検索機能には以下のような付加価値もあります:

- 検索ログの分析による利用動向の把握
- 関連性スコアの改善による検索精度の向上
- 検索インターフェースの改善によるユーザー体験の向上

このように、検索機能は情報システムにおいて非常に重要な役割を果たしています。検索の利点を最大限に活かすためには、適切なデータ構造、インデックス作成、ランキングアルゴリズムなどの工夫が必要不可欠です。

In [ ]:
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")

# 結論
このチュートリアルでは、Amazon Bedrock AgentCore Gateway とその組み込みの完全マネージド型セマンティック検索機能について学びました。以下の内容を確認しました。

- セマンティック検索を有効にしてゲートウェイを作成する方法
- 単一のエンドポイントから 300 以上の MCP ツールを提供するために、複数のゲートウェイターゲットを追加する方法
- 3 つの異なるアプローチを使用してゲートウェイ上のツールを一覧表示する方法
- 組み込みのセマンティック検索ツールを使用して関連するツールを見つける方法
- 検索を Strands Agent に統合する方法
- 数百のツールを持つサーバを使用するエージェントのパフォーマンスと、特定のトピックにツールを絞り込むためにセマンティック検索を使用するエージェントのパフォーマンスを比較する方法

AgentCore Gateway 検索は、より高度な使用例でも役立ちます。ネイティブの MCP ツールとしてではなく、コントロールプレーン API としてのみ検索を提供することで、エージェントに新しい MCP サーバーを検出し、実行時に新しい機能を見つけて、より難しい問題を解決するブレークスルーにつながる可能性があります。さらに、検索は MCP レジストリの重要な基盤であり、エージェント開発者が新しいエージェントを設計および構築する際の支援にもなります。

# リソースのクリーンアップ

以下は、テキストの英語から日本語への翻訳です。半角英数字の前後に半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。

When you are done using your resources, it is a good practice to clean them up. This will help you avoid unnecessary charges and keep your environment organized. Here are some steps to clean up your resources:

リソースの使用が終わったら、クリーンアップすることをお勧めします。これにより、不要な課金を避け、環境を整理することができます。リソースをクリーンアップするための手順は次のとおりです。

1. Delete the Cloud Storage bucket you created by running the following gsutil command:

   gsutil rm -r gs:// $BUCKET

作成した Cloud Storage バケットを削除するには、次の gsutil コマンドを実行します。

   gsutil rm -r gs:// $BUCKET

2. Delete the Cloud Pub/Sub topic by running the following gcloud command:

   gcloud pubsub topics delete $TOPIC

Cloud Pub/Sub トピックを削除するには、次の gcloud コマンドを実行します。

   gcloud pubsub topics delete $TOPIC

3. Delete the Cloud Function by running the following gcloud command:

   gcloud functions delete $FUNCTION_NAME

Cloud Function を削除するには、次の gcloud コマンドを実行します。

   gcloud functions delete $FUNCTION_NAME

まず、 AgentCore Gateway リソースをクリーンアップするためのヘルパー関数を定義しましょう。

In [ ]:
def delete_gatewaytarget(gateway_id):
    response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

    print(f"Found {len(response['items'])} targets for the gateway")

    for target in response["items"]:
        print(
            f"Deleting target with Name: {target['name']} and Id: {target['targetId']}"
        )

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target["targetId"]
        )
        time.sleep(20)


def delete_gateway(gateway_id):
    response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

### ゲートウェイターゲットの削除

日本語訳:

You can delete a gateway target using the AWS CLI or AWS CloudFormation. 

AWS CLI を使用してゲートウェイターゲットを削除するには、以下の delete-gateway-target コマンドを実行します。

```
aws apigatewayv2 delete-gateway-target \
    --gateway-id a1b2c3d4 \
    --target-id abcd1234
```

AWS CloudFormation を使用してゲートウェイターゲットを削除するには、AWS::ApiGatewayV2::GatewayTarget リソースを CloudFormation テンプレートから削除します。

ゲートウェイターゲットを削除すると、そのターゲットへのすべてのルートも削除されます。ゲートウェイターゲットを削除する前に、関連するルートを削除することをお勧めします。

In [ ]:
delete_gatewaytarget(gateway_id=gatewayId)

### ゲートウェイ自体の削除

日本語訳:

ゲートウェイ自体を削除するには、次の手順に従います。

1. `kubectl get deployments` を実行して、 `istio-ingressgateway` Deployment が存在することを確認します。

2. `istio-ingressgateway` Deployment を削除します。
   ```
   kubectl delete deployment istio-ingressgateway
   ```

3. 次に、 `istio-ingressgateway` Service を削除します。
   ```
   kubectl delete service istio-ingressgateway
   ```

4. 最後に、 `istio-ingressgateway` の Pod が削除されたことを確認します。
   ```
   kubectl get pods -l app=istio-ingressgateway
   ```

これで、ゲートウェイが完全に削除されました。

In [ ]:
delete_gateway(gateway_id=gatewayId)

In [ ]:
lambda_arns = [
    calc_lambda_resp["lambda_function_arn"],
    restaurant_lambda_resp["lambda_function_arn"],
]

for arn in lambda_arns:
    if utils.delete_gateway_lambda(arn):
        print(f"Deleted Lambda: {arn}")
    else:
        print(f"Lambda {arn} not found or deletion failed")

In [ ]:
# Gateway ロールのクリーンアップ

日本語訳:

この スクリプト は、古い Gateway ロール を削除し、新しい Gateway ロール を作成します。

最初に、既存の "Gateway" ロール を削除します。その後、新しい "Gateway" ロール を作成し、必要な権限 ( permissions ) を割り当てます。

注意: このスクリプトを実行する前に、影響を受ける可能性のあるリソースをよく確認してください。運用環境で実行する場合は、十分に注意してください。

使用方法:

```
$ terraform init
$ terraform apply
```

変数:

- `aws_region` - AWS リージョン ( デフォルト: us-east-1 )

出力:

- `gateway_role_arn` - 新しい Gateway ロールの ARN
if utils.delete_gateway_iam_role():
    print("Gateway IAM role deleted")
else:
    print("Gateway IAM role not found or deletion failed")

# Cognito のクリーンアップ

日本語訳:

Amazon Cognito は、Web や モバイルアプリケーションに認証、承認、ユーザー管理機能を簡単に追加できる AWS のサービスです。Cognito を使用すると、ユーザープールやアイデンティティプールを作成し、管理することができます。

時間が経つにつれて、これらのリソースが不要になる可能性があります。そのような場合は、AWS アカウントからそれらを削除する必要があります。これを行うには、以下の手順に従ってください。

1. AWS マネジメントコンソールにサインインします。
2. Cognito コンソールを開きます。
3. 削除するユーザープールまたはアイデンティティプールを選択します。
4. 「削除」ボタンをクリックします。
5. 確認ダイアログで「削除」を入力し、「削除」ボタンをクリックして削除を確定します。

注意: ユーザープールまたはアイデンティティプールを削除すると、そのリソースに関連付けられているすべてのデータが永久に失われます。この操作は元に戻せません。

Cognito のクリーンアップを行うことで、不要なリソースを削除し、AWS アカウントの請求額を削減できます。定期的にリソースを確認し、不要なものを削除することをお勧めします。
if utils.delete_cognito_user_pool():
    print("Cognito pool deleted")
else:
    print("✗ Failed to delete Cognito pool")